Begin to implement chains and implement a pipeline 

In [1]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [2]:
from util import chunk_markdown, cut_sections

In [3]:
sections = cut_sections(resume)

history = sections['history']['result']
chunks = chunk_markdown(history)

We need a function to only select job history from chunks. Any other headings such as summary, or education should be set aside.

## Begin to process job history chunks


Implementing `process_history_chain()`

In [ ]:
from beef import process_history_chain

chain = process_history_chain()
for chunk in chunks:
    chain({"section": chunk, "title": title, "desc": description})

How do we highlight key skills?

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from util import job_requirement_chain
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

requirement_chain = job_requirement_chain()
prompt = PromptTemplate(template="""
You're an expert career consultant with an IQ over 140 working with a special client regarding this job posting.
You will be given a list of required skills, and an excerpt from the client's resume.
What required 3 skills does the client have?

Resume excerpt:\n\t{section}

Required skills:\n\t{requirements}

{format_instructions}
""",
                        input_variables=['section', 'requirements'],
                        partial_variables={'format_instructions': format_instructions}
                                      )
key_skill_chain = LLMChain(prompt=prompt, llm=ChatOpenAI(temperature=.1, model_name="gpt-3.5-turbo"), output_parser=output_parser)
key_skill_chain.predict(section=chunks[0], requirements=requirement_chain.predict(description=description))

['Results-driven', 'Teamwork', 'Fast Learner']

In [ ]:
skills = _17

prompt = PromptTemplate.from_template("""
You will be given a job experience from a resume and 3 key skills.
Use bullet points to highlight how the job experience section demonstrates the given skills.

Skills: {skills}

Job Experience: {section} 
""")
# prompt = PromptTemplate.from_template("""
# You will be given a section of a resume and 3 key skills.
# Please write a one sentence summary for the resume section highlighting some or all of the given skills.
# 
# Skills: {skills}
# 
# Section: {section} 
# """)
highlight_chain = LLMChain(prompt=prompt, llm=ChatOpenAI(temperature=.1, model_name="gpt-3.5-turbo"))
highlight_chain.predict(section=chunks[0], skills=skills)

It seems that a router should be used to evaluate a job experience section. If a job experience section is not strong for a given job description, THAN the chains described in this notebook should be used. This chain should be called `BeefSectionChain`. It's intention should be to highlight the key skills needed for a given job description.